# Classifying Art Pieces Based on Style

## Description of the project

The goal of this project is to use deep learning in order to classify different art pieces based on style. I will use a TensorFlow's implementation of a CNN, as convolutional neural networks tend to produce good results when it comes to image classification in general. Keep in mind that many parts of the project are not done yet. Also, note that, in order to run the code provided here and the code that will come with the rest of the project, one will  naturally need to install TensorFlow on their system. The installation through *pip* was pretty simple and I just followed the instructions given by the book.

The dataset that I will be using for the project can be found on Kaggle, and will, of course, be included in the submission folder. The dataset is split into 2 parts: "dataset" & "museum_art", with "dataset" being larger and more comprehensive. For the project itself, I think that I will have more than enough images in the "dataset" directory, but if I end up thinking that there could be benefits to using both of those, that part shouldn't be too hard to change in the code later on in the project. Again, I think that there is enough material in the 'dataset' directory for the CNN to achieve fairly good results. Furthermore, both of those directories are split into 'training' and 'validation' sets, which is a nice feature provided by the person who created this dataset on Kaggle. I think that I will stick to that same split, but again, I am leaving myself some freedom here. Both the training and the validation portions are then split again, into directories corresponding to different classes: 'drawing', 'engraving', 'iconography', 'painting', 'sculpture'.

The first part of the project is definitely data processing. All of the image files need to be labeled correctly and then preprocessed using the features given to us by TensorFlow, specifically tensorflow.io and tensorflow.image modules.

Then comes the actual learning. Given what I have read online and in the book, there is a high chance that the algorithm will not perform well on the first try. Convolutonal neural networks, and deep neural networks in general, tend to overfit the data. Now since I am not planning on making an ensemble of multiple CNNs (that would be too complicated and computationally-intensive for what I'm trying to do), I will stick to the good old dropout to try to regularize the CNN a bit and improve its generalization property. This is an area for which I do have to do a bit more research, so particular ideas implementation-wise may change in these next few weeks.

The last part of the whole process will be testing and evaluation. In my Project Proposal assignment, I said that I was going to do k-fold cross validation for this. Now that I've done a lot of research about CNN's I think that this is unrealistic and impractical. As far I can tell from researching things online, k-fold cross validation is not often used with CNN's because there are simply too many parameters to look at. As far as the other, individual metrics, go, I think that I will mostly be concerned with accuracy. I put in my Project Proposal assignment that I would look at true positive and false negative rates, but even if I do that, it will definitely not be the focus of the project. Most of my time until the end of the semester will be spent in trying to make the network perform as best as it can, accuracy-wise.

## Status

It turns out that I was correct when I was initially writing this little introduction. A decent portion of the first part is done. I was able to familiarize myself with file paths in Python and use them nicely to properly name all the image files. I was also able to remove all the corrupted images that were initially making a lot of trouble when being provided as input to *tf.io.read_file* and *tf.image.decode_image*, which are the main two methods that I needed to use to process images using TensorFlow. The main question that remains to be answered for this part is how to effectively split the training data into baches that I will feed into the CNN itself. There are examples of this in the book and online and I am quite confident that it will not be tremendously hard to do. With that part done, this portion of the assignment should be done.

As far as other portions of the work go, the second and third part of the assignment are still remaining. I am hopeful that at the end of the following week, I will be pretty much done with the remaining parts of the data processing portion and with the learning portion, so that I can start brainstorming ideas that I could use to optimize this whole process further.

# Code

## Removing corrupt data and renaming files

Note that when I downloaded the dataset, all the files were named in a weird way and there was no convention present. I decided to name all the files in the following way. Each file will contain essentially three pieces of information. The first piece will tell us whether this file (image) is part of the training dataset or the testing dataset. The next part will tell us which class this particular file belongs to. The final part of the name will be the number of the image in the corresponding class.

And so an example of this naming convention would be:

_**training-drawing-123**_

This file would, of course, correspond to the 123rd .jpg file present in the training portion of the drawing class. Note that I am not including the extension of the file, as all of them are either going to be .jpg or .jpeg.

In [1]:
# Note that because of the image verification, it takes a while to run this code.

import os
import pathlib
import string
import PIL
from PIL import Image

# Forming the necessary paths
dataset_dir = pathlib.Path('.') / 'Data' / 'dataset'
museum_art_dir = pathlib.Path('.') / 'Data' / 'museum_art'

# Renaming individual files while traversing the file tree and removing any corrupt images
for directory in [dataset_dir, museum_art_dir]:
    for type_of_data in directory.iterdir():
        for class_label in type_of_data.iterdir():
            current_dir = class_label
            for index, image_file in enumerate(current_dir.iterdir()):
                try:
                    v_image = Image.open(image_file)
                    v_image.verify()
                    # If we pass these two checks means the image is not corrupt
                    old_file = current_dir / image_file.name
                    new_name = '{}-{}-{}'.format(type_of_data.name, class_label.name, index)
                    new_name += image_file.suffix
                    new_file = current_dir / new_name
                    # We also want to check whether we already created the given file
                    try:
                        os.rename(old_file, new_file)
                    except IOError as e:
                        pass
                except PIL.UnidentifiedImageError:
                    os.remove(image_file)
                
                

## Visualizing some representative images

I will use matplotlib to plot a few of the images from each class, just so that we can see what we're dealing with and so that I can practice using the TensorFlow API to process images. When I start using the Keras API, the specifics of processing images could change, but the important thing is that, even if I don't find a better way to do this, this one seems to be working just fine.

In [42]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pathlib
from PIL import Image

# Get the correct path
training_data = pathlib.Path('.') / 'Data' / 'dataset' / 'training'
drawing_path = training_data / 'drawing'
engraving_path = training_data / 'engraving'
iconography_path = training_data / 'iconography'
painting_path = training_data / 'painting'
sculpture_path = training_data / 'sculpture'

# Creating the corresponding lists
drawing_list = []
engraving_list = []
iconography_list = []
painting_list = []
sculpture_list = []

path_list = [drawing_path, engraving_path, iconography_path, painting_path, sculpture_path]

# Populate all the lists at once
for path in path_list:
    for index, image_file in enumerate(path.iterdir(), 1):
        if index > 10:
            break
        if path == path_list[0]:
            # Add to the list of drawings
            drawing_list.append(image_file)
        elif path == path_list[1]:
            # Add to the list of engravings
            engraving_list.append(image_file)
        elif path == path_list[2]:
            # Add to the list for iconography
            iconography_list.append(image_file)
        elif path == path_list[3]:
            # Add to the list of paintings
            painting_list.append(image_file)
        else:
            # Add to the list of sculptures
            sculpture_list.append(image_file)
            
# Define a function for plotting
def plot_images(image_list):
    figure = plt.figure(figsize=(20, 10))
    subplot_list = []
    for index, image_file in enumerate(image_list):
        # Have to use os.path.real_path here because
        # tf is complaining about pathlib.Path
        raw_image = tf.io.read_file(os.path.realpath(image_file))
        image = tf.image.decode_image(raw_image)
        ax = figure.add_subplot(2, 5, index+1)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(image)
        ax.set_title(image_file.name, size=15)
    plt.tight_layout()
    plt.show()

# Test out the function with the list of drawings
# plot_images(drawing_list)

# Same function can be used to show other images as well:
# plot_images(engraving_list)
# plot_images(iconography_list)
# plot_images(painting_list)
# plot_images(sculpture_list)

## Constructing data and connecting images to class labels

In this part, I will be trying to figure out a way to handle the size of my dataset, which is not small by any means. I will have to somehow split the training data into batches and then create tensors from those batches. Furthermore, I will have to construct tensors (or possibly even batches) of class labels for each tensor (or batch) of data that I'm using.

### Code for constructing TF Datasets from existing tensors

In [3]:
import tensorflow as tf
a = [1.2, 3.4, 7.5, 4.1, 5.0, 1.0]
# Constructing the dataset
ds = tf.data.Dataset.from_tensor_slices(a)

# Printing some values
for item in ds:
    print(item)

tf.Tensor(1.2, shape=(), dtype=float32)
tf.Tensor(3.4, shape=(), dtype=float32)
tf.Tensor(7.5, shape=(), dtype=float32)
tf.Tensor(4.1, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


### Code for constructing batches from datasets

In [4]:
# Constructing batches of size 3
ds_batch = ds.batch(3)

# Printing some values
for i, elem in enumerate(ds_batch, 1):
    print('batch {}:'.format(i), elem.numpy())

batch 1: [1.2 3.4 7.5]
batch 2: [4.1 5.  1. ]


### Code for combining two tensors into a joint dataset

In [9]:
# Creating two tensors
tf.random.set_seed(1)
t_x = tf.random.uniform([4, 3], dtype=tf.float32)
t_y = tf.range(4)

# Printing the two tensors
print(t_x)
print(t_y)

# Creating two datasets and then zipping them
ds_x = tf.data.Dataset.from_tensor_slices(t_x)
ds_y = tf.data.Dataset.from_tensor_slices(t_y)
ds_joint_1 = tf.data.Dataset.zip((ds_x, ds_y))

# Printing some output
for example in ds_joint_1:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

# Creating one, joint dataset
ds_joint_2 = tf.data.Dataset.from_tensor_slices((t_x, t_y))
for example in ds_joint_2:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

tf.Tensor(
[[0.16513085 0.9014813  0.6309742 ]
 [0.4345461  0.29193902 0.64250207]
 [0.9757855  0.43509948 0.6601019 ]
 [0.60489583 0.6366315  0.6144488 ]], shape=(4, 3), dtype=float32)
tf.Tensor([0 1 2 3], shape=(4,), dtype=int32)
 x: [0.16513085 0.9014813  0.6309742 ]  y: 0
 x: [0.4345461  0.29193902 0.64250207]  y: 1
 x: [0.9757855  0.43509948 0.6601019 ]  y: 2
 x: [0.60489583 0.6366315  0.6144488 ]  y: 3
 x: [0.16513085 0.9014813  0.6309742 ]  y: 0
 x: [0.4345461  0.29193902 0.64250207]  y: 1
 x: [0.9757855  0.43509948 0.6601019 ]  y: 2
 x: [0.60489583 0.6366315  0.6144488 ]  y: 3


### Code for batching, shuffling and repeating

In [36]:
# Creating a shuffled version of the ds_joint dataset
tf.random.set_seed(1)
ds = ds_joint_1.shuffle(buffer_size=len(t_x))
# for example in ds:
#     print(' x:', example[0].numpy(),
#           ' y:', example[1].numpy())

# Experimenting with batch function
ds = ds_joint_1.batch(batch_size=3,
                      drop_remainder=False)
for element in ds:
    x_part, y_part = element[0], element[1]
    print(x_part.numpy())
    print(y_part.numpy())
    print()

[[0.16513085 0.9014813  0.6309742 ]
 [0.4345461  0.29193902 0.64250207]
 [0.9757855  0.43509948 0.6601019 ]]
[0 1 2]

[[0.60489583 0.6366315  0.6144488 ]]
[3]



In [41]:
# Order 1: shuffle -> batch -> repeat
tf.random.set_seed(1)
ds = ds_joint_1.shuffle(4).batch(2).repeat(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())
print('\n\n')

# Order 2: batch -> shuffle -> repeat
tf.random.set_seed(1)
ds = ds_joint_1.batch(2).shuffle(4).repeat(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (2, 3) [2 1]
1 (2, 3) [0 3]
2 (2, 3) [0 3]
3 (2, 3) [1 2]
4 (2, 3) [3 0]
5 (2, 3) [1 2]



0 (2, 3) [0 1]
1 (2, 3) [2 3]
2 (2, 3) [0 1]
3 (2, 3) [2 3]
4 (2, 3) [2 3]
5 (2, 3) [0 1]


## Constructing the Training and Validation datasets

In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pathlib

# Get the correct paths
training_data = pathlib.Path('.') / 'Data' / 'dataset' / 'training'
validation_data = pathlib.Path('.') / 'Data' / 'dataset' / 'validation'

# Creating lists of training classes and validation classes
training_paths = []
validation_paths = []
for (train_path, valid_path) in zip(training_data.iterdir(), validation_data.iterdir()):
    training_paths.append(train_path)
    validation_paths.append(valid_path)

def get_label_from_path(path):
    if 'drawing' in str(path):
        return 0
    if 'engraving' in str(path):
        return 1
    if 'iconography' in str(path):
        return 2
    if 'painting' in str(path):
        return 3
    if 'sculpture' in str(path):
        return 4
    return None

def load_image_from_path(path, img_width=32, img_height=32):
    image = tf.io.read_file(str(path))
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    image /= 255.0
    return image

training_images_list = []
training_labels_list = []
for directory in training_paths:
    for file_path in directory.iterdir():
        training_images_list.append(load_image_from_path(file_path))
        training_labels_list.append(get_label_from_path(file_path))

# Construct the training dataset
training_images_ds = tf.data.Dataset.from_tensor_slices(training_images_list)
training_labels_ds = tf.data.Dataset.from_tensor_slices(training_labels_list)

validation_images_list = []
validation_labels_list = []
for directory in validation_paths:
    for file_path in directory.iterdir():
        validation_images_list.append(load_image_from_path(file_path))
        validation_labels_list.append(get_label_from_path(file_path))
        
# Construct the validation dataset
validation_images_ds = tf.data.Dataset.from_tensor_slices(validation_images_list)
validation_labels_ds = tf.data.Dataset.from_tensor_slices(validation_labels_list)

# Shuffling and batching the data
# Shuffle the training images and labels


In [6]:
training_images_list = tf.random.shuffle(training_images_list, seed=8)
training_labels_list = tf.random.shuffle(training_labels_list, seed=8)

## Trying to train a CNN

In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pathlib
from tensorflow.keras import datasets, layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(tf.convert_to_tensor(training_images_list), tf.convert_to_tensor(training_labels_list), epochs=20, 
                    validation_data=(tf.convert_to_tensor(validation_images_list), tf.convert_to_tensor(validation_labels_list)))

Epoch 1/20
242/242 [==============================] - 4s 18ms/step - loss: 0.9784 - accuracy: 0.6240 - val_loss: 0.7784 - val_accuracy: 0.7208
Epoch 2/20
242/242 [==============================] - 4s 18ms/step - loss: 0.7372 - accuracy: 0.7371 - val_loss: 0.6800 - val_accuracy: 0.7722
Epoch 3/20
242/242 [==============================] - 4s 18ms/step - loss: 0.6398 - accuracy: 0.7705 - val_loss: 0.6403 - val_accuracy: 0.7629
Epoch 4/20
242/242 [==============================] - 6s 25ms/step - loss: 0.5694 - accuracy: 0.7903 - val_loss: 0.6213 - val_accuracy: 0.7804
Epoch 5/20
242/242 [==============================] - 6s 25ms/step - loss: 0.5180 - accuracy: 0.8081 - val_loss: 0.5244 - val_accuracy: 0.8096
Epoch 6/20
242/242 [==============================] - 6s 25ms/step - loss: 0.4692 - accuracy: 0.8296 - val_loss: 0.5757 - val_accuracy: 0.7874
Epoch 7/20
242/242 [==============================] - 6s 25ms/step - loss: 0.4213 - accuracy: 0.8420 - val_loss: 0.5895 - val_accuracy: 0.7815